In [1]:
pip install pydub pyannote.audio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 41.5 MB/s eta 0:00:0

In [1]:
from pydub import AudioSegment
import os
from pyannote.audio import Pipeline
from pyannote.audio.pipelines.utils.hook import ProgressHook
import torch
import pandas as pd

In [ ]:
#from pydub import AudioSegment
#import os

def mp3_to_wav(audio_file_path):
  '''
  coverts mp3 file to wav file
  '''
  sound = AudioSegment.from_mp3(audio_file_path)
  wav_file_path = os.path.splitext(audio_file_path)[0] + ".wav"
  sound.export(wav_file_path, format="wav")
  return wav_file_path


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `S2 project token` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `S2 proje

In [3]:
#from pyannote.audio import Pipeline

# Loading pyannote model
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.yaml:   0%|          | 0.00/469 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/399 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/26.6M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/221 [00:00<?, ?B/s]

In [4]:
# To run on GPU

pipeline.to(torch.device("cuda"))

In [5]:
def diarization_segment(diarization_results):
  '''
  coverts diarization results to segments
  '''
  segments = []

  for turn, _, speaker in diarization_results.itertracks(yield_label=True):
    segments.append({
        "start": round(turn.start, 2),
        "end": round(turn.end, 2),
        "speaker": speaker
    })

  return segments

In [ ]:
# Convert mp3 to wav
# don't need for now
mp3_to_wav('DS330203.mp3')

'DS330203.wav'

In [13]:
#from payannote.audio.pipelines.utils.hook import ProgressHook

# Run the model on the whole file
with ProgressHook() as hook:
  diarization_result = pipeline("Data.wav", hook=hook)

Output()

In [14]:
# Converting diarization result to segment
diarization_segments = diarization_segment(diarization_result)

# Coverting to DataFrame
diarization_df = pd.DataFrame(diarization_segments)

# Save DataFrame
diarization_df.to_csv('pyannote_without_SN.csv')

In [15]:
diarization_df

,start,end,speaker
0,1.92,2.39,SPEAKER_00
1,4.86,5.48,SPEAKER_00
2,7.52,7.64,SPEAKER_02
3,7.98,8.23,SPEAKER_02
4,8.23,8.43,SPEAKER_00
5,8.43,8.57,SPEAKER_02
6,14.00,14.04,SPEAKER_02
7,14.07,14.32,SPEAKER_02
8,15.30,15.39,SPEAKER_01
9,16.52,16.64,SPEAKER_01


 Bad result by not specifying the number of speakers. The model detected 5 speakers.

## With specifying the number of speakers

In [16]:
# Run the model on the whole file with specifying number of speaker
with ProgressHook() as hook:
  diarization_result = pipeline("Data.wav", num_speakers=2, hook=hook)

Output()

In [17]:
# Converting diarization result to segment
diarization_segments = diarization_segment(diarization_result)

# Coverting to DataFrame
diarization_df = pd.DataFrame(diarization_segments)

# Save DataFrame
diarization_df.to_csv('pyannote_with_SN.csv')

In [18]:
diarization_df

,start,end,speaker
0,1.92,2.39,SPEAKER_00
1,4.86,5.48,SPEAKER_00
2,7.52,7.64,SPEAKER_01
3,7.98,8.23,SPEAKER_01
4,8.23,8.43,SPEAKER_00
5,8.43,8.57,SPEAKER_01
6,14.00,14.04,SPEAKER_01
7,14.07,14.32,SPEAKER_01
8,15.30,15.39,SPEAKER_01
9,16.52,17.18,SPEAKER_01
